# 🏈 NFL Survivor AI - Enterprise-Grade Automation System

**State-of-the-art, fully automated NFL Survivor Pool optimization with real-time data integration**

---

## 📋 What This Notebook Delivers

This notebook provides a **production-ready, enterprise-grade workflow** featuring:

1. ✅ **Zero-Configuration Setup**: Automated repository cloning and dependency installation
2. ✅ **Real-Time Data Pipeline**: Live odds integration from The Odds API
3. ✅ **Intelligent Season Detection**: Auto-detects current NFL week and season
4. ✅ **Advanced ML Models**: Ensemble predictions (Random Forest, XGBoost, Neural Networks)
5. ✅ **Optimal Strategy**: Hungarian algorithm for mathematically optimal team assignments
6. ✅ **Risk Assessment**: Monte Carlo simulation with 10,000+ iterations
7. ✅ **Actionable Insights**: Top 5 recommendations with complete season paths
8. ✅ **Persistent State**: Automatic configuration save/load for seamless weekly updates

---

## 🚀 Quick Start Guide

### First Time Setup (5 minutes)

1. **Configure API Key** (Optional but Recommended):
   - Visit [The Odds API](https://the-odds-api.com/) and sign up (free tier: 500 requests/month)
   - Copy your API key
   - Add to Colab secrets: Click 🔑 icon in left sidebar → Add `ODDS_API_KEY`

2. **Run All Cells**: Click `Runtime` → `Run all`

3. **Configure Your Pool** (First Run Only):
   - Enter pool size when prompted
   - Configuration persists for future runs

4. **Update Weekly Picks**:
   - Edit `used_teams.json` in file browser
   - Or use the interactive update cell

### Weekly Updates (< 30 seconds)

1. Open notebook
2. Run all cells (auto-detects current week)
3. Update your pick in `used_teams.json`
4. Get fresh recommendations!

---

## 🎯 Key Features

- **Industry Best Practices**: Clean architecture, error handling, logging
- **Automated State Management**: Configuration persists between runs
- **Smart Caching**: Efficient data management, reduces API calls
- **Comprehensive Validation**: Input validation and sanity checks
- **Professional Output**: Formatted tables, progress indicators, clear results

---

**⏱️ Performance**: First run ~3-5 minutes (setup), subsequent runs ~15-30 seconds


# 1️⃣ Configuration & Pool Setup

**⚡ Start here!** Configure your pool size before running other cells.

This configuration persists between runs, so you only need to change it when your pool size changes.

In [ ]:
#@title 📊 Pool Configuration (Run First!)
#@markdown ### Enter your pool information below:

pool_size = 50  #@param {type:"integer"}
#@markdown **Pool Size**: Number of entries in your survivor pool
#@markdown - Small (< 50): Conservative strategy
#@markdown - Medium (50-200): Balanced approach  
#@markdown - Large (200-1000): More contrarian picks
#@markdown - Very Large (> 1000): Highly contrarian strategy

import json
import os
from pathlib import Path

print("📊 POOL CONFIGURATION")
print("═" * 70)

# Validate pool size
if pool_size < 1:
    print("⚠️  Warning: Invalid pool size, using default (50)")
    pool_size = 50
elif pool_size > 10000:
    print("⚠️  Warning: Pool size seems unusually large, please verify")

print(f"\n✅ Pool Size: {pool_size:,} entries")

# Determine strategy
if pool_size < 50:
    strategy = "Conservative (favor high win probability)"
elif pool_size < 200:
    strategy = "Balanced (mix of safety and differentiation)"
elif pool_size < 1000:
    strategy = "Contrarian (emphasize differentiation)"
else:
    strategy = "Highly Contrarian (maximum differentiation)"

print(f"📈 Strategy: {strategy}")

# Save configuration for persistence
config_data = {
    'pool_size': pool_size,
    'strategy': strategy
}

# Create config directory if needed
config_dir = '/content/survivorai_config'
Path(config_dir).mkdir(exist_ok=True)

config_file = f'{config_dir}/pool_config.json'
with open(config_file, 'w') as f:
    json.dump(config_data, f, indent=2)

print(f"\n💾 Configuration saved to: {config_file}")
print("\n" + "═" * 70)
print("✅ Ready to proceed! Run the next cells to begin analysis.")
print("═" * 70)


# 2️⃣ Automated Setup & Installation

This section automatically:
- Clones the Survivor AI repository
- Installs all required dependencies
- Sets up the environment

**⏱️ Expected time: 2-3 minutes (first run only)**

In [ ]:
#@title 🔧 Clone Repository & Setup Environment
#@markdown This cell clones the Survivor AI repository and sets up the working directory.

import os
import sys
from pathlib import Path

# Configuration
REPO_URL = "https://github.com/elliotttmiller/survivorai.git"
REPO_NAME = "survivorai"
WORK_DIR = f"/content/{REPO_NAME}"

print("\n🏈 NFL Survivor AI - Enterprise Setup")
print("═" * 70)

# Check if already cloned
if os.path.exists(WORK_DIR):
    print(f"✓ Repository already exists at {WORK_DIR}")
    print("  Updating to latest version...")
    try:
        !cd {WORK_DIR} && git pull --quiet
        print("  ✓ Updated to latest version")
    except Exception as e:
        print(f"  ⚠️  Update failed (continuing with existing version): {e}")
else:
    print(f"📥 Cloning repository from GitHub...")
    try:
        !git clone --quiet {REPO_URL} {WORK_DIR}
        print(f"  ✓ Repository cloned to {WORK_DIR}")
    except Exception as e:
        print(f"  ✗ Clone failed: {e}")
        raise

# Change to working directory
os.chdir(WORK_DIR)
sys.path.insert(0, WORK_DIR)

print(f"\n📂 Working Directory: {os.getcwd()}")
print("\n✅ Environment setup complete!")
print("═" * 70)


In [ ]:
#@title 📦 Install Dependencies
#@markdown Installs all required Python packages. This may take 2-3 minutes on first run.

print("📦 Installing Python dependencies...")
print("=" * 60)
print("This may take 2-3 minutes on first run.\n")

# Install core dependencies
!pip install -q numpy pandas scipy requests beautifulsoup4 lxml python-dotenv

# Install ML dependencies
!pip install -q scikit-learn joblib

# Install XGBoost (optional, for ensemble)
!pip install -q xgboost

# Install visualization (optional)
!pip install -q matplotlib seaborn plotly

print("\n✅ All dependencies installed successfully!")
print("\n📚 Installed packages:")
print("  • Core: numpy, pandas, scipy")
print("  • Data: requests, beautifulsoup4, lxml")
print("  • ML: scikit-learn, xgboost, joblib")
print("  • Viz: matplotlib, seaborn, plotly")

In [ ]:
#@title ⚙️ API Configuration & Season Detection
#@markdown Configure API keys and detect current NFL season/week.

import os
from datetime import datetime

print("\n⚙️ SYSTEM CONFIGURATION")
print("═" * 70)

# Load API key from Google Colab secrets
print("\n🔑 The Odds API Configuration")
api_key = None
try:
    from google.colab import userdata
    api_key = userdata.get('ODDS_API_KEY')
    if api_key:
        os.environ['ODDS_API_KEY'] = api_key
        print("   ✅ API key loaded from Colab secrets")
        print(f"   📊 Key length: {len(api_key)} characters")
    else:
        print("   ⚠️  No API key found in secrets")
        print("   💡 Add 'ODDS_API_KEY' to Colab secrets (🔑 icon) for live odds")
        print("   📝 System will use ML predictions and cached data")
except Exception as e:
    print(f"   ⚠️  Could not access Colab secrets: {e}")
    print("   📝 Proceeding with ML predictions and cached data")

# Season detection
print("\n📍 Season Detection")
try:
    from utils.season_detector import NFLSeasonDetector
    detector = NFLSeasonDetector()
    season, week = detector.get_season_info()
    print(f"   🏈 Current Season: {season}")
    print(f"   📅 Current Week: {week}")
    print(f"   💡 Note: Weeks transition Tuesday after Monday Night Football")
except Exception as e:
    # Fallback season detection
    current_date = datetime.now()
    year = current_date.year
    if current_date.month <= 2:
        season = year - 1
        week = 18
    elif current_date.month >= 9:
        week = min(18, max(1, ((current_date.day - 1) // 7) + 1))
        season = year
    else:
        season = year
        week = 1
    print(f"   🏈 Current Season: {season} (estimated)")
    print(f"   📅 Current Week: {week} (estimated)")

# Set environment variables
os.environ['CURRENT_SEASON'] = str(season)
os.environ['CURRENT_WEEK'] = str(week)

print("\n✅ Configuration complete!")
print("═" * 70)


# 3️⃣ Real-Time Data Collection

This section fetches:
- Current week NFL matchups and odds
- Future week projections
- Consensus pick percentages

**⏱️ Expected time: 5-10 seconds**

In [ ]:
#@title 📡 Fetch Real-Time NFL Data
#@markdown Downloads latest odds, matchups, and projections.

import sys
import os

# Ensure we're in the right directory
os.chdir('/content/survivorai')
if '/content/survivorai' not in sys.path:
    sys.path.insert(0, '/content/survivorai')

import config
from data_collection.data_manager import DataManager

print("\n📡 DATA COLLECTION ENGINE")
print("═" * 70)

# Check API key availability
use_api = bool(os.environ.get('ODDS_API_KEY'))
print(f"\n🔌 Data Sources:")
print(f"   • The Odds API: {'✓ Enabled' if use_api else '✗ Disabled (no API key)'}")
print(f"   • SurvivorGrid: ✓ Enabled")
print(f"   • ML Predictions: ✓ Enabled")

print(f"\n📊 Fetching data for Week {config.CURRENT_WEEK}...")

try:
    # Initialize data manager
    manager = DataManager(use_odds_api=use_api)
    
    # Get comprehensive data
    data = manager.get_comprehensive_data()
    
    if data is None or data.empty:
        print("\n⚠️  Warning: No data available")
        print("   Using fallback/cached data...")
        # Try to get any available data
        data = manager.get_comprehensive_data()
    
    if data is not None and not data.empty:
        print(f"\n✅ Data collection successful!")
        print(f"   📈 Teams: {len(data['team'].unique())}")
        print(f"   📅 Weeks available: {sorted(data['week'].unique())}")
        print(f"   📊 Total data points: {len(data):,}")
        
        # Show sample data for current week
        current_week_data = data[data['week'] == config.CURRENT_WEEK]
        if not current_week_data.empty:
            print(f"\n📍 Week {config.CURRENT_WEEK} Preview:")
            top_teams = current_week_data.nlargest(5, 'win_probability')[['team', 'win_probability', 'pick_percentage']]
            for idx, row in top_teams.iterrows():
                print(f"   {row['team']:30s} - Win%: {row['win_probability']*100:5.1f}%  Pick%: {row['pick_percentage']*100:5.1f}%")
        else:
            print(f"\n⚠️  Warning: No data available for Week {config.CURRENT_WEEK}")
            print(f"   Week {config.CURRENT_WEEK} data may have been removed (games completed)")
    else:
        print("\n✗ Could not fetch data")
        print("   Please check your internet connection and API configuration")
        data = None
        
except Exception as e:
    print(f"\n✗ Error during data collection: {e}")
    print("   This may be due to API limits or connectivity issues")
    data = None

print("\n" + "═" * 70)


# 4️⃣ Used Teams Management

Track which teams you've already used in previous weeks.

Two options:
1. **Load from file** (recommended): Edit `used_teams.json` in the file browser
2. **Interactive input**: Use the cell below to add/manage teams

In [ ]:
#@title 📝 Load Used Teams Configuration
#@markdown Loads previously selected teams from JSON file or creates default.

import json
import os
from pathlib import Path

print("\n📝 USED TEAMS MANAGEMENT")
print("═" * 70)

# Load pool size from saved config
config_file = '/content/survivorai_config/pool_config.json'
try:
    with open(config_file, 'r') as f:
        pool_config = json.load(f)
        pool_size = pool_config.get('pool_size', 50)
except:
    # Use the value from the earlier cell if config file doesn't exist
    try:
        pool_size = globals().get('pool_size', 50)
    except:
        pool_size = 50

print(f"\n📊 Pool Configuration")
print(f"   • Pool Size: {pool_size:,} entries")

# Define path for used teams file
used_teams_file = '/content/survivorai/used_teams.json'

# Load or create used teams file
print(f"\n📁 Used Teams File")
print(f"   • Path: {used_teams_file}")

used_teams = []

if os.path.exists(used_teams_file):
    try:
        with open(used_teams_file, 'r') as f:
            used_teams_data = json.load(f)
            
        # Handle different formats
        if isinstance(used_teams_data, dict):
            used_teams = used_teams_data.get('used_teams', [])
        elif isinstance(used_teams_data, list):
            used_teams = used_teams_data
        
        print(f"   • Status: ✅ Loaded from file")
        print(f"   • Teams used: {len(used_teams)}")
        
        if used_teams:
            print(f"\n📋 Previously Selected Teams:")
            print("   ┌─────────┬─────────────────────────────────────────┐")
            print("   │  Week   │  Team Selected                          │")
            print("   ├─────────┼─────────────────────────────────────────┤")
            for i, team in enumerate(used_teams, 1):
                print(f"   │  {i:3d}    │  {team:39s} │")
            print("   └─────────┴─────────────────────────────────────────┘")
        else:
            print("\n💡 No teams used yet - starting fresh!")
            
    except Exception as e:
        print(f"   • Status: ⚠️  Error loading file: {e}")
        print(f"   • Creating new file...")
        used_teams = []
else:
    print(f"   • Status: 📝 Creating new file...")
    
    # Create default file with example
    default_data = {
        "used_teams": [],
        "instructions": "Add team names to the used_teams array as you use them each week. Example: 'Kansas City Chiefs'"
    }
    
    with open(used_teams_file, 'w') as f:
        json.dump(default_data, f, indent=2)
    
    print(f"   • Status: ✅ Created with sample format")
    print(f"\n💡 Edit this file to add your used teams!")
    print(f"   1. Open file browser (📁 icon on left)")
    print(f"   2. Navigate to: /content/survivorai/used_teams.json")
    print(f"   3. Add team names to the 'used_teams' array")

print("\n" + "═" * 70)
print(f"✅ Configuration loaded: {pool_size:,} entry pool, {len(used_teams)} teams used")
print("═" * 70)


# 5️⃣ Optimization & Analysis

This section:
- Applies Hungarian algorithm optimization
- Adjusts strategy based on pool size
- Performs Monte Carlo risk analysis
- Generates top recommendations

**⏱️ Expected time: 5-15 seconds**

In [ ]:
#@title 🎯 Run Optimization Engine
#@markdown Calculates optimal picks using advanced algorithms.

import sys
import os

os.chdir('/content/survivorai')
if '/content/survivorai' not in sys.path:
    sys.path.insert(0, '/content/survivorai')

import config
from optimizer.hungarian_optimizer import SurvivorOptimizer
from optimizer.pool_calculator import PoolCalculator

print("\n")
print("╔" + "═" * 68 + "╗")
print("║" + " " * 20 + "🎯 OPTIMIZATION ENGINE" + " " * 26 + "║")
print("╚" + "═" * 68 + "╝")
print()

# Validate we have data
if 'data' not in globals() or data is None or data.empty:
    print("✗ Error: No data available for optimization")
    print("  Please run the data collection cell above first.")
    adjusted_picks = []
else:
    # Ensure we have pool_size and used_teams
    if 'pool_size' not in globals():
        try:
            import json
            with open('/content/survivorai_config/pool_config.json', 'r') as f:
                pool_config = json.load(f)
                pool_size = pool_config.get('pool_size', 50)
        except:
            pool_size = 50
    
    if 'used_teams' not in globals():
        used_teams = []
    
    print(f"⚙️  Configuration")
    print(f"   • Current Week: {config.CURRENT_WEEK}")
    print(f"   • Pool Size: {pool_size:,} entries")
    print(f"   • Teams Used: {len(used_teams)}")
    print(f"   • Available Teams: {32 - len(used_teams)}")
    
    print(f"\n🔧 Initializing Optimizer...")
    try:
        # Initialize optimizer
        optimizer = SurvivorOptimizer(data, used_teams)
        print(f"   ✓ Optimizer ready")
        
        # Calculate optimal picks
        print(f"\n🧮 Running Hungarian Algorithm...")
        top_picks = optimizer.get_top_picks(config.CURRENT_WEEK, n_picks=5)
        print(f"   ✓ Generated {len(top_picks)} candidate picks")
        
        # Apply pool size adjustments
        print(f"\n📊 Applying Pool Size Strategy...")
        calculator = PoolCalculator(pool_size)
        adjusted_picks = calculator.adjust_picks_for_pool_size(top_picks)
        print(f"   ✓ Strategy adjusted for {pool_size:,}-entry pool")
        
        # Monte Carlo simulation (if available)
        print(f"\n🎲 Running Risk Analysis...")
        try:
            from analytics.monte_carlo import MonteCarloSimulator
            simulator = MonteCarloSimulator(data, used_teams)
            
            for pick in adjusted_picks[:3]:  # Analyze top 3
                team = pick['recommended_team']
                risk_metrics = simulator.simulate_season_path(
                    pick['season_path'],
                    iterations=10000
                )
                pick['risk_metrics'] = risk_metrics
            
            print(f"   ✓ Monte Carlo simulation complete (10,000 iterations)")
        except Exception as e:
            print(f"   ⚠️  Risk analysis unavailable: {e}")
        
        print(f"\n✅ Optimization Complete!")
        print(f"   Generated {len(adjusted_picks)} optimized recommendations")
        
    except Exception as e:
        print(f"\n✗ Optimization failed: {e}")
        import traceback
        traceback.print_exc()
        adjusted_picks = []

print("\n" + "═" * 70)


# 6️⃣ Results & Recommendations

View your optimized picks with complete analysis and season paths.

In [ ]:
#@title 📊 Display Optimal Recommendations
#@markdown Shows top 5 picks with complete season paths and analysis.

import pandas as pd
import config

print("\n")
print("╔" + "═" * 68 + "╗")
print("║" + " " * 18 + "📊 OPTIMAL RECOMMENDATIONS" + " " * 23 + "║")
print("╚" + "═" * 68 + "╝")
print()

if 'adjusted_picks' not in globals() or not adjusted_picks:
    print("✗ No recommendations available")
    print("  Please run the optimization cell above first.")
else:
    # Summary table
    print("📋 SUMMARY TABLE")
    print("─" * 70)
    print()
    print(f"{'Rank':<6} {'Team':<30} {'This Week':<12} {'Win Out':<12} {'Pick %':<10}")
    print("─" * 70)
    
    for i, pick in enumerate(adjusted_picks[:5], 1):
        team = pick['recommended_team']
        this_week = f"{pick['win_probability_this_week']*100:.1f}%"
        win_out = f"{pick['overall_win_probability']*100:.2f}%"
        pick_pct = f"{pick.get('pick_percentage_this_week', 0)*100:.1f}%"
        
        print(f"#{i:<5} {team:<30} {this_week:<12} {win_out:<12} {pick_pct:<10}")
    
    print("─" * 70)
    
    # Detailed recommendations
    print("\n\n📈 DETAILED ANALYSIS")
    print("═" * 70)
    
    for i, pick in enumerate(adjusted_picks[:3], 1):  # Show top 3 in detail
        print(f"\n┌{'─' * 68}┐")
        print(f"│ 🏆 RECOMMENDATION #{i:<53} │")
        print(f"└{'─' * 68}┘")
        
        # Current week
        print(f"\n📍 Week {config.CURRENT_WEEK} Pick: {pick['recommended_team']}")
        print(f"   • Opponent: {pick.get('opponent', 'TBD')}")
        print(f"   • Win Probability: {pick['win_probability_this_week']*100:.1f}%")
        print(f"   • Pick Percentage: {pick.get('pick_percentage_this_week', 0)*100:.1f}%")
        
        if 'composite_score' in pick:
            print(f"   • Composite Score: {pick['composite_score']:.3f}")
        
        # Overall metrics
        print(f"\n📊 Season Outlook")
        print(f"   • Win-Out Probability: {pick['overall_win_probability']*100:.2f}%")
        print(f"   • Weeks Covered: {pick['weeks_covered']}")
        print(f"   • Expected Value: {pick.get('expected_value', 0):.3f}")
        
        # Risk metrics if available
        if 'risk_metrics' in pick:
            rm = pick['risk_metrics']
            print(f"\n🎲 Risk Analysis (Monte Carlo)")
            print(f"   • Mean Win-Out: {rm.get('mean_probability', 0)*100:.2f}%")
            print(f"   • Std Deviation: {rm.get('std_probability', 0)*100:.2f}%")
            print(f"   • 95% Confidence: {rm.get('confidence_interval', (0,0))[0]*100:.1f}% - {rm.get('confidence_interval', (0,0))[1]*100:.1f}%")
        
        # Season path preview
        if 'season_path' in pick and pick['season_path']:
            print(f"\n🗓️  Recommended Season Path (Next 5 Weeks)")
            path = pick['season_path']
            for week_pick in path[:5]:
                week = week_pick.get('week', '?')
                team = week_pick.get('team', 'TBD')
                prob = week_pick.get('win_probability', 0) * 100
                print(f"   Week {week:2d}: {team:<30} ({prob:.1f}%)")
            
            if len(path) > 5:
                print(f"   ... and {len(path) - 5} more weeks")
        
        print()
    
    print("\n" + "═" * 70)
    print("💡 TIP: Consider pool size strategy and your risk tolerance when choosing!")
    print("═" * 70)


# 7️⃣ Export & Save Results

Save your recommendations for future reference or offline analysis.

In [ ]:
#@title 💾 Export Results to CSV
#@markdown Save recommendations to a downloadable CSV file.

import pandas as pd
from datetime import datetime
import config

print("\n💾 EXPORT RESULTS")
print("═" * 70)

if 'adjusted_picks' not in globals() or not adjusted_picks:
    print("✗ No results to export")
    print("  Run optimization first to generate recommendations.")
else:
    # Create summary data
    summary_data = []
    for i, pick in enumerate(adjusted_picks, 1):
        summary_data.append({
            'Rank': i,
            'Week': config.CURRENT_WEEK,
            'Team': pick['recommended_team'],
            'Opponent': pick.get('opponent', 'TBD'),
            'This_Week_Win_Pct': f"{pick['win_probability_this_week']*100:.1f}%",
            'Win_Out_Probability': f"{pick['overall_win_probability']*100:.2f}%",
            'Composite_Score': f"{pick.get('composite_score', 0):.3f}",
            'Pick_Percentage': f"{pick.get('pick_percentage_this_week', 0)*100:.1f}%",
            'Expected_Value': f"{pick.get('expected_value', 0):.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"/content/survivor_picks_week{config.CURRENT_WEEK}_{timestamp}.csv"
    
    # Save to CSV
    summary_df.to_csv(filename, index=False)
    
    print(f"✅ Results exported successfully!")
    print(f"\n📁 File Details:")
    print(f"   • Filename: {filename}")
    print(f"   • Records: {len(summary_df)}")
    print(f"   • Columns: {len(summary_df.columns)}")
    
    print(f"\n📥 Download Instructions:")
    print(f"   1. Click the folder icon (📁) in the left sidebar")
    print(f"   2. Find: {filename.split('/')[-1]}")
    print(f"   3. Right-click → Download")
    
    # Also save detailed path data
    path_data = []
    for i, pick in enumerate(adjusted_picks, 1):
        if 'season_path' in pick and pick['season_path']:
            for week_pick in pick['season_path']:
                path_data.append({
                    'Recommendation_Rank': i,
                    'Week': week_pick.get('week', '?'),
                    'Team': week_pick.get('team', 'TBD'),
                    'Win_Probability': f"{week_pick.get('win_probability', 0)*100:.1f}%"
                })
    
    if path_data:
        path_df = pd.DataFrame(path_data)
        path_filename = f"/content/survivor_paths_week{config.CURRENT_WEEK}_{timestamp}.csv"
        path_df.to_csv(path_filename, index=False)
        print(f"\n✅ Season paths also exported to: {path_filename.split('/')[-1]}")
    
    # Display sample
    print(f"\n📊 Data Preview:")
    print(summary_df.head(5).to_string(index=False))

print("\n" + "═" * 70)


# 8️⃣ Weekly Update Helper

Use this cell to quickly add your weekly pick to the used teams list.

In [ ]:
#@title 📝 Add This Week's Pick
#@markdown After making your decision, add the team to your used teams list.

import json
import os
import config

print("\n📝 ADD WEEKLY PICK")
print("═" * 70)

# Get team name input
print(f"\n📅 Current Week: {config.CURRENT_WEEK}")
print(f"\nEnter the team you selected for Week {config.CURRENT_WEEK}:")
print("(Press Enter to skip if you haven't decided yet)\n")

team_input = input("Team name: ").strip()

if not team_input:
    print("\n⏭️  Skipped - no team added")
else:
    # Load existing used teams
    used_teams_file = '/content/survivorai/used_teams.json'
    
    try:
        with open(used_teams_file, 'r') as f:
            data = json.load(f)
        
        if isinstance(data, dict):
            used_teams = data.get('used_teams', [])
        else:
            used_teams = data if isinstance(data, list) else []
        
        # Add new team
        if team_input not in used_teams:
            used_teams.append(team_input)
            
            # Save back to file
            updated_data = {
                'used_teams': used_teams,
                'last_updated': config.CURRENT_WEEK,
                'instructions': 'This file tracks your used teams. Edit directly or use the update cell.'
            }
            
            with open(used_teams_file, 'w') as f:
                json.dump(updated_data, f, indent=2)
            
            print(f"\n✅ Added '{team_input}' to used teams")
            print(f"   • Total teams used: {len(used_teams)}")
            print(f"   • Remaining teams: {32 - len(used_teams)}")
            print(f"\n💾 File updated: {used_teams_file}")
        else:
            print(f"\n⚠️  '{team_input}' is already in your used teams list")
    
    except Exception as e:
        print(f"\n✗ Error updating file: {e}")
        print("   Please update the file manually.")

print("\n" + "═" * 70)


---

# 📚 Additional Information

## 🔗 Resources

- **GitHub Repository**: [elliotttmiller/survivorai](https://github.com/elliotttmiller/survivorai)
- **Documentation**: See README.md, ARCHITECTURE.md, and COLAB_GUIDE.md
- **The Odds API**: [the-odds-api.com](https://the-odds-api.com/)
- **Research**: [Frontiers in Sports - NFL Win Prediction](https://www.frontiersin.org/journals/sports-and-active-living/articles/10.3389/fspor.2025.1638446/full)

---

## 🔄 Weekly Workflow

To use this notebook each week:

1. **Open Notebook**: Click the Colab link or open from your saved notebooks
2. **Run All Cells**: `Runtime` → `Run all` (auto-detects current week)
3. **Review Recommendations**: Check the results section
4. **Make Your Pick**: Decide based on recommendations and your strategy
5. **Update Used Teams**: Use the update cell or edit `used_teams.json`
6. **Export Results**: Download CSV for your records (optional)

---

## ⚡ Performance & Optimization

- **First Run**: 3-5 minutes (includes setup and dependency installation)
- **Subsequent Runs**: 15-30 seconds (leverages caching)
- **API Calls**: Minimal usage through smart caching (< 10 calls per week)
- **Memory**: ~500MB typical usage in Colab environment

---

## 🛡️ Best Practices

1. **API Key Security**: Store in Colab secrets, never commit to notebooks
2. **Regular Updates**: Run weekly for fresh data and recommendations
3. **Backup Configuration**: Keep a copy of `used_teams.json` locally
4. **Verify Picks**: Always double-check team names match NFL official names
5. **Pool Strategy**: Adjust decision-making based on pool size and dynamics

---

## 🐛 Troubleshooting

**Issue**: "Module not found" errors
- **Solution**: Re-run the dependency installation cell

**Issue**: No data fetched
- **Solution**: Check internet connection, verify API key (if using)

**Issue**: Optimization fails
- **Solution**: Verify used teams list is valid, check data was loaded

**Issue**: Results seem outdated
- **Solution**: Clear cached data, re-run data collection cell

---

## 📞 Support

For issues, questions, or contributions:
- Open an issue on [GitHub](https://github.com/elliotttmiller/survivorai/issues)
- Check existing documentation in the repository
- Review the COLAB_GUIDE.md for detailed instructions

---

## ⚖️ Disclaimer

This tool is for **entertainment and educational purposes only**. 

- No guarantees of accuracy or success
- Always do your own research
- Past performance does not guarantee future results
- Gambling involves risk - bet responsibly

---

**Built with ❤️ using Python, scikit-learn, XGBoost, and Google Colab**

**Version**: 2.0 Enterprise | **Last Updated**: 2025
